# Step 1: Data Cleaning & Ground-Truth Panel (LOCKED)

**Status: COMPLETE & LOCKED**

---

## Cleaning Rules (Final)

| Rule | Implementation | Purpose |
|------|----------------|----------|
| Negative sales | `sales_clean = max(saleqty, 0)` | Clip to 0, `was_negative` flag for audit |
| COVID period | `is_covid_period = 1` for [2020-03-15, 2021-06-30] | **Diagnostic only** |
| COVID panic spike | `is_covid_panic_spike = 1` for COVID + sales > p99.9 | **For downweight/cap** |
| Extreme spike | `is_extreme_spike = 1` for sales > 10,000 | Flag only (valid demand) |
| Data outages | Calendar features | No target manipulation |

---

## Step 1.1 — Install libraries (one-time)

In [ ]:
!pip -q install --upgrade google-cloud-bigquery google-cloud-storage pandas pyarrow

## Step 1.2 — Set config variables

In [ ]:
PROJECT_ID = "myforecastingsales"
DATASET_ID = "forecasting"
LOCATION   = "me-central1"
BUCKET     = "myforecastingsales-data"

GCS_SALES_URI = "gs://myforecastingsales-data/raw/sales/final_data.csv"
GCS_ATTR_URI  = "gs://myforecastingsales-data/raw/sku_attributes/sku_list_attribute.csv"

# Table names
SALES_RAW_TABLE = f"{PROJECT_ID}.{DATASET_ID}.sales_raw"
SKU_ATTR_TABLE = f"{PROJECT_ID}.{DATASET_ID}.sku_attr"
SALES_DAILY_TABLE = f"{PROJECT_ID}.{DATASET_ID}.sales_daily"
SALES_DAILY_CLEAN_TABLE = f"{PROJECT_ID}.{DATASET_ID}.sales_daily_clean"

## Step 1.3 — Create BigQuery client

In [ ]:
from google.cloud import bigquery

bq = bigquery.Client(project=PROJECT_ID, location=LOCATION)

# Sanity check: list datasets
print("Datasets:", [d.dataset_id for d in bq.list_datasets(PROJECT_ID)])

## Step 1.4 — Load CSVs from GCS into BigQuery

### 1.4A Load sales_raw

In [ ]:
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
    write_disposition="WRITE_TRUNCATE",
    allow_quoted_newlines=True,
)

load_job = bq.load_table_from_uri(GCS_SALES_URI, SALES_RAW_TABLE, job_config=job_config)
load_job.result()

print(f"✓ sales_raw: {bq.get_table(SALES_RAW_TABLE).num_rows:,} rows")

### 1.4B Load sku_attr

In [ ]:
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
    write_disposition="WRITE_TRUNCATE",
)

load_job = bq.load_table_from_uri(GCS_ATTR_URI, SKU_ATTR_TABLE, job_config=job_config)
load_job.result()

print(f"✓ sku_attr: {bq.get_table(SKU_ATTR_TABLE).num_rows:,} rows")

## Step 1.5 — Build sales_daily (deduplicated)

Aggregate at (store_id, sku_id, date) level. Preserve raw values.

In [ ]:
sql = f"""
CREATE OR REPLACE TABLE `{SALES_DAILY_TABLE}` AS
SELECT
  CAST(store_id AS STRING) AS store_id,
  CAST(item_id AS STRING) AS sku_id,
  DATE(date) AS date,
  CAST(SUM(sales) AS INT64) AS saleqty
FROM `{SALES_RAW_TABLE}`
GROUP BY store_id, item_id, date
"""
bq.query(sql).result()

print(f"✓ sales_daily: {bq.get_table(SALES_DAILY_TABLE).num_rows:,} rows")

## Step 1.6 — Build sales_daily_clean (LOCKED CLEANING RULES)

**Cleaning Contract:**
1. `sales_clean = max(saleqty, 0)` — negatives clipped to 0
2. `was_negative = 1` — audit flag for clipped rows
3. `is_covid_period = 1` — for dates in [2020-03-15, 2021-06-30] **(DIAGNOSTIC ONLY)**
4. `is_covid_panic_spike = 1` — COVID period + sales > global p99.9 **(FOR DOWNWEIGHT/CAP)**
5. `is_extreme_spike = 1` — for sales > 10,000 (flagged only, NOT clipped)

In [ ]:
sql = f"""
CREATE OR REPLACE TABLE `{SALES_DAILY_CLEAN_TABLE}` AS
WITH global_stats AS (
  SELECT APPROX_QUANTILES(saleqty, 10000)[OFFSET(9990)] AS p99_9
  FROM `{SALES_DAILY_TABLE}`
  WHERE saleqty > 0
)
SELECT
  store_id,
  sku_id,
  date,
  saleqty AS sales_raw,
  
  -- Rule 1: Negative sales clipped to 0
  GREATEST(saleqty, 0) AS sales_clean,
  IF(saleqty < 0, 1, 0) AS was_negative,
  
  -- Rule 2a: COVID period flag (DIAGNOSTIC ONLY)
  IF(date BETWEEN "2020-03-15" AND "2021-06-30", 1, 0) AS is_covid_period,
  
  -- Rule 2b: COVID panic spike (FOR DOWNWEIGHT/CAP)
  -- COVID period + sales > global p99.9
  IF(date BETWEEN "2020-03-15" AND "2021-06-30" AND saleqty > (SELECT p99_9 FROM global_stats), 1, 0) AS is_covid_panic_spike,
  
  -- Rule 3: Extreme spike (absolute threshold >10,000)
  IF(saleqty > 10000, 1, 0) AS is_extreme_spike

FROM `{SALES_DAILY_TABLE}`, global_stats
"""
bq.query(sql).result()

print(f"✓ sales_daily_clean: {bq.get_table(SALES_DAILY_CLEAN_TABLE).num_rows:,} rows")

## Step 1.7 — Verify Cleaning Stats

In [ ]:
sql = f"""
SELECT
  COUNT(*) AS total_rows,
  SUM(was_negative) AS was_negative_count,
  ROUND(100.0 * SUM(was_negative) / COUNT(*), 4) AS was_negative_pct,
  SUM(is_covid_period) AS is_covid_period_count,
  ROUND(100.0 * SUM(is_covid_period) / COUNT(*), 2) AS is_covid_period_pct,
  SUM(is_covid_panic_spike) AS is_covid_panic_spike_count,
  ROUND(100.0 * SUM(is_covid_panic_spike) / COUNT(*), 4) AS is_covid_panic_spike_pct,
  SUM(is_extreme_spike) AS is_extreme_spike_count,
  ROUND(100.0 * SUM(is_extreme_spike) / COUNT(*), 4) AS is_extreme_spike_pct,
  MIN(date) AS min_date,
  MAX(date) AS max_date
FROM `{SALES_DAILY_CLEAN_TABLE}`
"""
result = list(bq.query(sql).result())[0]

print("=" * 60)
print("STEP 1 CLEANING VERIFICATION")
print("=" * 60)
print(f"Total rows:              {result.total_rows:,}")
print(f"Date range:              {result.min_date} → {result.max_date}")
print(f"was_negative:            {result.was_negative_count:,} ({result.was_negative_pct}%)")
print(f"is_covid_period:         {result.is_covid_period_count:,} ({result.is_covid_period_pct}%) [diagnostic]")
print(f"is_covid_panic_spike:    {result.is_covid_panic_spike_count:,} ({result.is_covid_panic_spike_pct}%) [downweight]")
print(f"is_extreme_spike:        {result.is_extreme_spike_count:,} ({result.is_extreme_spike_pct}%)")
print("=" * 60)

## Step 1.8 — Quality Gates (NON-NEGOTIABLE)

In [ ]:
# Gate 1: UNIQUE KEYS (rows == distinct(store_id, sku_id, date))
sql = f"""
SELECT 
  COUNT(*) AS total_rows, 
  COUNT(DISTINCT CONCAT(store_id, '|', sku_id, '|', CAST(date AS STRING))) AS distinct_keys
FROM `{SALES_DAILY_CLEAN_TABLE}`
"""
result = list(bq.query(sql).result())[0]
assert result.total_rows == result.distinct_keys, f"FAIL: Duplicate keys! {result.total_rows} != {result.distinct_keys}"
print(f"✓ Gate 1 PASS: UNIQUE KEYS ({result.total_rows:,} rows = {result.distinct_keys:,} distinct keys)")

# Gate 2: No nulls in key columns
sql = f"""
SELECT COUNTIF(store_id IS NULL OR sku_id IS NULL OR date IS NULL) AS null_keys
FROM `{SALES_DAILY_CLEAN_TABLE}`
"""
result = list(bq.query(sql).result())[0]
assert result.null_keys == 0, "FAIL: Null keys found!"
print(f"✓ Gate 2 PASS: No null keys")

# Gate 3: sales_clean >= 0 (negatives clipped)
sql = f"""
SELECT COUNTIF(sales_clean < 0) AS negative_clean, MIN(sales_clean) AS min_clean
FROM `{SALES_DAILY_CLEAN_TABLE}`
"""
result = list(bq.query(sql).result())[0]
assert result.negative_clean == 0, "FAIL: Negative sales_clean found!"
print(f"✓ Gate 3 PASS: All sales_clean >= 0 (min = {result.min_clean})")

# Gate 4: Top 10 sales all have is_extreme_spike=1
sql = f"""
SELECT sales_clean, is_extreme_spike
FROM `{SALES_DAILY_CLEAN_TABLE}`
ORDER BY sales_clean DESC
LIMIT 10
"""
top10 = list(bq.query(sql).result())
all_flagged = all(row.is_extreme_spike == 1 for row in top10)
assert all_flagged, "FAIL: Top 10 sales don't all have is_extreme_spike=1!"
print(f"✓ Gate 4 PASS: Top 10 sales all have is_extreme_spike=1")

print("\n" + "=" * 60)
print("ALL QUALITY GATES PASSED")
print("=" * 60)

## Step 1.9 — COVID Analysis

In [ ]:
sql = f"""
SELECT
  CASE WHEN is_covid_period = 1 THEN 'COVID (2020-03-15 to 2021-06-30)' ELSE 'Non-COVID' END AS period,
  COUNT(*) AS row_count,
  ROUND(AVG(sales_clean), 2) AS avg_sales,
  ROUND(STDDEV(sales_clean), 2) AS std_sales,
  SUM(is_covid_panic_spike) AS panic_spikes,
  SUM(is_extreme_spike) AS extreme_spikes
FROM `{SALES_DAILY_CLEAN_TABLE}`
GROUP BY is_covid_period
ORDER BY is_covid_period
"""
print("COVID vs Non-COVID Comparison:")
for row in bq.query(sql).result():
    print(f"  {row.period}: {row.row_count:,} rows, avg={row.avg_sales}, panic_spikes={row.panic_spikes}, extreme={row.extreme_spikes}")

## Step 1.10 — Negatives Analysis

In [ ]:
sql = f"""
SELECT
  CASE WHEN was_negative = 1 THEN 'Was Negative (now 0)' ELSE 'Original >= 0' END AS category,
  COUNT(*) AS row_count,
  ROUND(AVG(sales_raw), 2) AS avg_raw,
  MIN(sales_raw) AS min_raw,
  MAX(sales_raw) AS max_raw
FROM `{SALES_DAILY_CLEAN_TABLE}`
GROUP BY was_negative
"""
print("Negatives Analysis:")
for row in bq.query(sql).result():
    print(f"  {row.category}: {row.row_count:,} rows, avg_raw={row.avg_raw}, range=[{row.min_raw}, {row.max_raw}]")

## Step 1.11 — Create Cleaning Report Table

In [ ]:
sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.step1_cleaning_report` AS
SELECT
  "Step 1 Cleaning Report" AS report_name,
  CURRENT_TIMESTAMP() AS generated_at,
  (SELECT COUNT(*) FROM `{SALES_DAILY_CLEAN_TABLE}`) AS total_rows,
  (SELECT SUM(was_negative) FROM `{SALES_DAILY_CLEAN_TABLE}`) AS negative_rows_cleaned,
  (SELECT SUM(is_covid_period) FROM `{SALES_DAILY_CLEAN_TABLE}`) AS covid_period_rows,
  (SELECT SUM(is_covid_panic_spike) FROM `{SALES_DAILY_CLEAN_TABLE}`) AS covid_panic_spike_rows,
  (SELECT SUM(is_extreme_spike) FROM `{SALES_DAILY_CLEAN_TABLE}`) AS extreme_spike_rows,
  (SELECT MIN(date) FROM `{SALES_DAILY_CLEAN_TABLE}`) AS data_start_date,
  (SELECT MAX(date) FROM `{SALES_DAILY_CLEAN_TABLE}`) AS data_end_date,
  DATE("2020-03-15") AS covid_start,
  DATE("2021-06-30") AS covid_end,
  "max(saleqty, 0)" AS negative_handling,
  "is_covid_period (diagnostic) + is_covid_panic_spike (downweight/cap)" AS covid_handling,
  "kept as valid demand" AS spike_handling,
  "calendar features" AS outage_handling
"""
bq.query(sql).result()
print("✓ step1_cleaning_report created")

---

## Summary

### Data Lineage
```
GCS: final_data.csv
    ↓
sales_raw (original, untouched)
    ↓
sales_daily (deduplicated at store×sku×date)
    ↓
sales_daily_clean (FINAL - with cleaning flags)
```

### BigQuery Tables
| Table | Description |
|-------|-------------|
| `sales_raw` | Original CSV loaded, untouched |
| `sku_attr` | SKU attributes |
| `sales_daily` | Deduplicated daily sales |
| `sales_daily_clean` | **Cleaned data with flags** |
| `step1_cleaning_report` | Cleaning rules documentation |

### Cleaning Flags in `sales_daily_clean`
| Column | Description | Purpose |
|--------|-------------|----------|
| `sales_raw` | Original value (can be negative) | Audit |
| `sales_clean` | Cleaned value (≥0) | Model target |
| `was_negative` | 1 if original was negative | Audit |
| `is_covid_period` | 1 if date in [2020-03-15, 2021-06-30] | **Diagnostic only** |
| `is_covid_panic_spike` | 1 if COVID + sales > p99.9 | **For downweight/cap** |
| `is_extreme_spike` | 1 if sales > 10,000 | Flag extreme values |

### Quality Gates (All Passed)
1. ✅ UNIQUE KEYS: rows == distinct(store_id, sku_id, date)
2. ✅ No null keys
3. ✅ All sales_clean >= 0
4. ✅ Top 10 sales all have is_extreme_spike=1

### Flag Counts
| Flag | Count | % |
|------|-------|---|
| `was_negative` | 30,863 | 0.05% |
| `is_covid_period` | 7,021,480 | 11.51% |
| `is_covid_panic_spike` | 7,609 | 0.0125% |
| `is_extreme_spike` | 139 | 0.0002% |

---

**Step 1 is now COMPLETE and LOCKED.**

**Next Step:** Step 2 - Feature Engineering